In [1]:
!python -V

Python 3.12.11


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [6]:
%%bash
if [ ! -f "../data/green_tripdata_2021-01.parquet" ]; then
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet -P ../data/
fi
if [ ! -f "../data/green_tripdata_2021-02.parquet" ]; then
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet -P ../data/
fi

In [7]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/06/29 19:29:03 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/29 19:29:03 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='file:c:/Users/Daniel/Documents/mlops-zoomcamp-epam/02-experiment-tracking/mlruns/1', creation_time=1751095781303, experiment_id='1', last_update_time=1751095781303, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
# modified for parquet use
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715205672146

In [15]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Daniel")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle")

In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

c:\Users\Daniel\Documents\mlops-zoomcamp-epam\.venv\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\Daniel\Documents\mlops-zoomcamp-epam\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [19:30:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.72668                          
[1]	validation-rmse:11.27886                          
[2]	validation-rmse:10.86706                          
[3]	validation-rmse:10.48887                          
[4]	validation-rmse:10.14190                          
[5]	validation-rmse:9.82428                           
[6]	validation-rmse:9.53413                           
[7]	validation-rmse:9.26922                           
[8]	validation-rmse:9.02790                           
[9]	validation-rmse:8.80814                           
[10]	validation-rmse:8.60824                          
[11]	validation-rmse:8.42667                          
[12]	validation-rmse:8.26226                          
[13]	validation-rmse:8.11328                          
[14]	validation-rmse:7.97797                          
[15]	validation-rmse:7.85583                          
[16]	validation-rmse:7.74518                          
[17]	validation-rmse:7.64509                          
[18]	valid

c:\Users\Daniel\Documents\mlops-zoomcamp-epam\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [19:32:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.67251                                                      
[1]	validation-rmse:11.18041                                                      
[2]	validation-rmse:10.73176                                                      
[3]	validation-rmse:10.32364                                                      
[4]	validation-rmse:9.95370                                                       
[5]	validation-rmse:9.61886                                                       
[6]	validation-rmse:9.31573                                                       
[7]	validation-rmse:9.03964                                                       
[8]	validation-rmse:8.79441                                                       
[9]	validation-rmse:8.57071                                                       
[10]	validation-rmse:8.37107                                                      
[11]	validation-rmse:8.18958                                                      
[12]

c:\Users\Daniel\Documents\mlops-zoomcamp-epam\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [19:33:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:10.80986                                                     
[1]	validation-rmse:9.73403                                                      
[2]	validation-rmse:8.91803                                                      
[3]	validation-rmse:8.30846                                                      
[4]	validation-rmse:7.85050                                                      
[5]	validation-rmse:7.51984                                                      
[6]	validation-rmse:7.27387                                                      
[7]	validation-rmse:7.10473                                                      
[8]	validation-rmse:6.96836                                                      
[9]	validation-rmse:6.87252                                                      
[10]	validation-rmse:6.79860                                                     
[11]	validation-rmse:6.74794                                                     
[12]	validation-

c:\Users\Daniel\Documents\mlops-zoomcamp-epam\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [19:34:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.57735                                                   
[1]	validation-rmse:11.00472                                                   
[2]	validation-rmse:10.49054                                                   
[3]	validation-rmse:10.03053                                                   
[4]	validation-rmse:9.61882                                                    
[5]	validation-rmse:9.25223                                                    
[6]	validation-rmse:8.92625                                                    
[7]	validation-rmse:8.63538                                                    
[8]	validation-rmse:8.37797                                                    
[9]	validation-rmse:8.15130                                                    
[10]	validation-rmse:7.95021                                                   
[11]	validation-rmse:7.77291                                                   
[12]	validation-rmse:7.61718            

KeyboardInterrupt: 

In [22]:
mlflow.xgboost.autolog(disable=True)

In [23]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

c:\Users\Daniel\Documents\mlops-zoomcamp-epam\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [20:17:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

2025/06/29 20:18:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Daniel\Documents\mlops-zoomcamp-epam\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [20:18:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/06/29 20:18:49 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/06/29 20:18:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
